# Import

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

## Load data
Data text annotated with data stance 100 pro and 100 kontra.

In [ ]:
df_user_stance = pd.read_csv("/content/drive/MyDrive/Data Science Project/Social Graph PPKM/labeled_data.csv")
df_user_stance = df_user_stance[["user_screen_name","user_name","in_reply_to_screen_name","text","stance"]]
df_user_stance

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_screen_name,user_name,in_reply_to_screen_name,text,stance
0,RustamIbrahim,Voter Education,NaN,1) tentu kita setuju saja dengan pengawasan pr...,pro
1,Anjaswisesa_K,ANJAS WISESA,kumparan,@kumparan kalo liat berita luar negeri namanya...,kontra
2,mhsgunadarma,IG : Mahasiswagunadarma,NaN,pemberlakuan pembatasan kegiatan masyarakat al...,pro
3,KATADATAcoid,Katadata.co.id,NaN,ketua satuan tugas penanganan covid-19 doni mo...,pro
4,Beritasatu,BeritaSatu,NaN,menko perekonomian tegaskan ppkm berbeda denga...,pro
...,...,...,...,...,...
92494,P3rsada27,🅿🅴🆁🆂🅰🅳🅰 👉🏻2⃣7⃣🇲🇨,NaN,"kasus covid-19 melandai berkat penerapan ppkm,...",NaN
92495,P3rsada27,🅿🅴🆁🆂🅰🅳🅰 👉🏻2⃣7⃣🇲🇨,NaN,penerapan ppkm untuk mengendalikan laju covid-...,NaN
92496,KangSemproel,Hilik Ku Aink Wae Lah✌,mapunk121,@mapunk121 @endridok itu mah pengalaman adek g...,NaN
92497,ariestarico2,Riest,crnnamon,@iecstyling dukung ppkm,NaN


In [ ]:
df_training = df_user_stance.iloc[:200]
df_training.head()

,user_screen_name,user_name,in_reply_to_screen_name,text,stance
0,RustamIbrahim,Voter Education,NaN,1) tentu kita setuju saja dengan pengawasan pr...,pro
1,Anjaswisesa_K,ANJAS WISESA,kumparan,@kumparan kalo liat berita luar negeri namanya...,kontra
2,mhsgunadarma,IG : Mahasiswagunadarma,NaN,pemberlakuan pembatasan kegiatan masyarakat al...,pro
3,KATADATAcoid,Katadata.co.id,NaN,ketua satuan tugas penanganan covid-19 doni mo...,pro
4,Beritasatu,BeritaSatu,NaN,menko perekonomian tegaskan ppkm berbeda denga...,pro


In [ ]:
df_testing = df_user_stance.iloc[200:]
df_testing.head()

,user_screen_name,user_name,in_reply_to_screen_name,text,stance
200,SunandarPS,Sunandar Siap diVaksin,NaN,satgas covid-19: kebijakan ppkm upaya menjamin...,NaN
201,KATADATAcoid,Katadata.co.id,NaN,"asosiasi ojek online mengatakan, pendapatan me...",NaN
202,mus_theib,DEMOKRAT NO SALE,penk_kehidupan,@penk_kehidupan tutup ppkm/psbb,NaN
203,MangSiHejo,Mang Kopi,NaN,"hari ketiga ppkm, positif covid-19 di subang y...",NaN
204,apriandhys,"Karev, A.",NaN,kenapa ga jam malam secara ketat aja?\n\njam 2...,NaN


# Pre-processing

In [ ]:
import re
def clean_data(data):
  description = data.lower()
  description = re.sub(r'^b\'', '', description)
  description = re.sub(r'^rt ', '', description)
  description = re.sub(r'\\x.{2}', '', description)
  description = re.sub(r'((www\.[^\s]*)|(https?://[^\s]*))', '', description)
  description = re.sub(r'\s+', ' ', description)
  description = description.strip()
  normal_regex = re.compile(r"(.)\1{1,}")
  description = normal_regex.sub(r"\1\1", description)
  description = re.sub(r'@[^\s]+', '', description)
  description = re.sub(r'#[^\s]+', '', description)
  description = re.sub(r'[^\w\s]', '', description)
  description = re.sub(r'\d+', ' ', description) 
  return description

In [ ]:
import nltk 
nltk.download("stopwords") 
nltk.download('punkt')
from nltk.corpus import stopwords 

!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xEIQwYre1SR71uRdQuez9MDmAUIysvAG' -O stopwordsID.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=19NOzXA8Voturopg_DTuiMns3s4M2IAUz' -O kamus_singkatan.csv
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VjgivEr1pxyRCuyhVifPnaReFz0yd8Us' -O colloquial-indonesian-lexicon.csv

stopwordsID = pd.read_csv("stopwordsID.csv", header=None)
df_kamus_singkatan = pd.read_csv('kamus_singkatan.csv')
df_kamus_alay = pd.read_csv('colloquial-indonesian-lexicon.csv')

#remove stopwords
def remove_stopword(data):
  token = nltk.word_tokenize(data)
  token_new = []
  for k in token:
    if k in df_kamus_singkatan['singkatan'].values:
        k = df_kamus_singkatan.loc[df_kamus_singkatan['singkatan']
                                    == k, 'asli'].values[0]
    if k in df_kamus_alay['slang'].values:
        k = df_kamus_alay.loc[df_kamus_alay['slang']
                              == k, 'formal'].values[0]
    if k not in stopwordsID[0].values and k not in stopwords.words("english"):
        token_new.append(k)

  str_clean = ' '.join(token_new)
  return str_clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Apply Pre-Processing

In [ ]:
text_list = []
texts = df_training.text

for text in texts:
    text = clean_data(text)
    text = remove_stopword(text)
    text_list.append(text)

text_list[:5]

['setuju pengawasan protokol kesehatan ketat daerah jawa bali istilah psbb psbb transisi rem darurat ppkm dan sebagainya penularan berkurang dki dan lain lain meningkat',
 'lihat berita negeri namanya lockdown new normal psbb totalpsbb transisipsbb parsialpsbb proporsional ppkm alah istilah efektif',
 'pemberlakuan pembatasan kegiatan masyarakat alias ppkm istilah psbb istilahnya biar corona bingung minggat',
 'ketua satuan tugas penanganan covid doni monardo berharap aktif corona diturunkan penerapan pembatasan kegiatan masyarakat ppkm wilayah jawa bali',
 'menko perekonomian tegaskan ppkm berbeda psbb']

In [ ]:
text_list_2 = []
texts = df_testing.text

for text2 in texts:
    text2 = clean_data(text2)
    text2 = remove_stopword(text2)
    # text2 = stemming(text2)
    text_list_2.append(text2)

text_list_2[:5]

['satgas covid kebijakan ppkm upaya menjamin keselamatan masyarakat',
 'asosiasi ojek online pendapatan membaik psbb diperlonggar ppkm garda penghasilan anggota turun',
 'tutup ppkmpsbb',
 'ketiga ppkm positif covid subang jalani isolasi capai orang',
 'jam malam ketat jam pakai alarm kayak film purge mencekam ngelanggar kena sanksi pidana jera sanksi formalitas psbbppkm']

# TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
train_text = vectorizer.fit_transform(text_list).todense()

In [ ]:
test_text = vectorizer.transform(text_list_2).todense()

# Classifier

## SVM

In [ ]:
# Import svm model
from sklearn import svm

clf = svm.SVC(kernel='linear') # Linear Kernel
clf.fit(train_text,df_training["stance"] )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


SVC(kernel='linear')

In [ ]:
from sklearn.metrics import classification_report
y_pred_clf = clf.predict(test_text)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


# Data testing prediction

In [ ]:
df_testing['stance'] = y_pred_clf
df_testing.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,user_screen_name,user_name,in_reply_to_screen_name,text,stance
200,SunandarPS,Sunandar Siap diVaksin,NaN,satgas covid-19: kebijakan ppkm upaya menjamin...,pro
201,KATADATAcoid,Katadata.co.id,NaN,"asosiasi ojek online mengatakan, pendapatan me...",kontra
202,mus_theib,DEMOKRAT NO SALE,penk_kehidupan,@penk_kehidupan tutup ppkm/psbb,kontra
203,MangSiHejo,Mang Kopi,NaN,"hari ketiga ppkm, positif covid-19 di subang y...",pro
204,apriandhys,"Karev, A.",NaN,kenapa ga jam malam secara ketat aja?\n\njam 2...,kontra


In [ ]:
merge_df = pd.concat([df_training, df_testing], axis=0)

In [ ]:
merge_df

,user_screen_name,user_name,in_reply_to_screen_name,text,stance
0,RustamIbrahim,Voter Education,NaN,1) tentu kita setuju saja dengan pengawasan pr...,pro
1,Anjaswisesa_K,ANJAS WISESA,kumparan,@kumparan kalo liat berita luar negeri namanya...,kontra
2,mhsgunadarma,IG : Mahasiswagunadarma,NaN,pemberlakuan pembatasan kegiatan masyarakat al...,pro
3,KATADATAcoid,Katadata.co.id,NaN,ketua satuan tugas penanganan covid-19 doni mo...,pro
4,Beritasatu,BeritaSatu,NaN,menko perekonomian tegaskan ppkm berbeda denga...,pro
...,...,...,...,...,...
92494,P3rsada27,🅿🅴🆁🆂🅰🅳🅰 👉🏻2⃣7⃣🇲🇨,NaN,"kasus covid-19 melandai berkat penerapan ppkm,...",pro
92495,P3rsada27,🅿🅴🆁🆂🅰🅳🅰 👉🏻2⃣7⃣🇲🇨,NaN,penerapan ppkm untuk mengendalikan laju covid-...,pro
92496,KangSemproel,Hilik Ku Aink Wae Lah✌,mapunk121,@mapunk121 @endridok itu mah pengalaman adek g...,kontra
92497,ariestarico2,Riest,crnnamon,@iecstyling dukung ppkm,pro


In [ ]:
dict = {}

for index, row in merge_df.iterrows():
  if row['user_screen_name'] in dict.keys():
    if row['stance'] == 'pro':
      dict[row['user_screen_name']]['pro'] += 1
    else:
      dict[row['user_screen_name']]['kontra'] += 1
  else:
    if row['stance'] == 'pro':
      dict[row['user_screen_name']] = {'pro':1,'kontra':0,'in_reply':[]}
    else:
      dict[row['user_screen_name']] = {'pro':0,'kontra':1,'in_reply':[]}

df_unique_stance = pd.DataFrame(columns = ['user_screen_name', 'stance'])

for key, value in dict.items():
  st = ''
  if value['pro'] > value['kontra']:
    st = 'pro'
  elif value['pro'] < value['kontra']:
    st = 'kontra'    
  else:
    st = '-'
  df_unique_stance = df_unique_stance.append({'user_screen_name' : key, 'stance' : st,}, ignore_index = True)

In [ ]:
df_unique_stance

,user_screen_name,stance
0,RustamIbrahim,kontra
1,Anjaswisesa_K,kontra
2,mhsgunadarma,kontra
3,KATADATAcoid,pro
4,Beritasatu,pro
...,...,...
18788,dykookieluv,kontra
18789,yntsmntr,kontra
18790,onyurmrk,kontra
18791,KwanKhong,kontra


In [ ]:
data_user = pd.read_csv("/content/drive/MyDrive/Data Science Project/Social Graph PPKM/user_with_gender.csv")

In [ ]:
data_user["stance"]  = df_unique_stance["stance"]
data_user

,Unnamed: 0,user_screen_name,user_name,gender,stance
0,0,RustamIbrahim,Voter Education,m,kontra
1,1,Anjaswisesa_K,ANJAS WISESA,m,kontra
2,2,nieke_aereta,Olivia Nieke Yuliane,w,kontra
3,3,ferizandra,ferizandra,w,pro
4,4,gloriamadona,Gloria Hermawan,w,pro
...,...,...,...,...,...
18788,18788,dykookieluv,dy⁷ kookie 🍪💜,w,kontra
18789,18789,yntsmntr,unyun 🍒,w,kontra
18790,18790,onyurmrk,fan ◡̈ ia,m,kontra
18791,18791,KwanKhong,dimitri,w,kontra


In [ ]:
data_user = data_user[data_user.stance != "-"]

In [ ]:
data_user

,Unnamed: 0,user_screen_name,user_name,gender,stance
0,0,RustamIbrahim,Voter Education,m,kontra
1,1,Anjaswisesa_K,ANJAS WISESA,m,kontra
2,2,nieke_aereta,Olivia Nieke Yuliane,w,kontra
3,3,ferizandra,ferizandra,w,pro
4,4,gloriamadona,Gloria Hermawan,w,pro
...,...,...,...,...,...
18788,18788,dykookieluv,dy⁷ kookie 🍪💜,w,kontra
18789,18789,yntsmntr,unyun 🍒,w,kontra
18790,18790,onyurmrk,fan ◡̈ ia,m,kontra
18791,18791,KwanKhong,dimitri,w,kontra


In [ ]:
data_user.to_csv('/content/drive/MyDrive/Data Science Project/master_data_gender_stance.csv')